# 추출된 의상의 색상을 파악하는 코드

* 이전에 학습된 상의, 하의 segment 모델에서 색상을 추출하여 확인 하는 코드를 구현

In [4]:
import os

print(os.getcwd())

/home/gon/dev_ws/deeplearning-repo-4/test/cloth_color


In [26]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np

# 색상 범위 정의
color_ranges = {
    'red1': [(0, 100, 100), (10, 255, 255)],
    'red2': [(170, 100, 100), (180, 255, 255)],
    'orange': [(10, 100, 100), (25, 255, 255)],
    'yello': [(25, 100, 100), (35, 255, 255)],
    'green': [(35, 100, 100), (85, 255, 255)],
    'blue': [(85, 100, 100), (125, 255, 255)],
    'navy': [(125, 100, 100), (140, 255, 255)],
    'violet': [(140, 100, 100), (170, 255, 255)],
    'white': [(0, 0, 125), (180, 30, 255)],
    'gray': [(0, 0, 70), (180, 30, 125)],
    'black': [(0, 0, 1), (180, 50, 70)]
}

# YOLOv8 모델 불러오기
model = YOLO('best_clothes_seg.pt')  # Segmentation 모델 경로
# 카메라 열기
cap = cv2.VideoCapture(0)  # 0은 기본 카메라
# 실시간 영상 처리 루프
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("카메라에서 영상을 읽을 수 없습니다.")
        break
    # YOLOv8 모델로 추론
    results = model.predict(source=frame, iou=0.25)
    # 추론된 segmentation을 영상에 적용
    for result in results:
        # Segmentation 마스크 그리기
        if result.masks is not None:
            for mask in result.masks.data:
                mask = mask.cpu().numpy()  # Mask를 numpy 배열로 변환
                colored_mask = (mask * 255).astype('uint8')  # Mask를 컬러화
                colored_mask = cv2.cvtColor(colored_mask, cv2.COLOR_GRAY2BGR)
                # Segmentation 마스크가 적용된 영역의 색상 추출
                masked_area = frame[mask.astype(bool)]  # 마스크가 적용된 부분만 선택
                
                # mask 데이터를 frame bit 연산이 되도록 boot type 에서 uint8 type 으로 변환 후 masking 처리
                mask255 = (mask * 255).astype("uint8")
                masked_frame = cv2.bitwise_and(frame, frame, mask=mask255)

                hsv_masked_area = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2HSV)
                
                # 각 색상 범위에 맞는 마스크 생성
                color_areas = {}
                for color_name, (lower, upper) in color_ranges.items():
                    lower_bound = np.array(lower, dtype=np.uint8)
                    upper_bound = np.array(upper, dtype=np.uint8)
                    color_mask = cv2.inRange(hsv_masked_area, lower_bound, upper_bound)

                    # 마스킹 된 영역의 픽셀수 계산
                    area_size = cv2.countNonZero(color_mask)
                    color_areas[color_name] = area_size

                largest_color = max(color_areas, key=color_areas.get)   
                largest_color_value = color_areas[largest_color]

                color_areas.pop(largest_color)

                next_largest_color = max(color_areas, key=color_areas.get)
                next_largest_color_value = color_areas[next_largest_color]

                show_color = largest_color

                if largest_color_value < next_largest_color_value * 3.5:
                    show_color = show_color + " and " + next_largest_color

                # 마스크를 원본 영상에 적용
                frame = cv2.addWeighted(frame, 1, colored_mask, 0, 0)  # Segmentation 마스크를 영상에 적용
                # 마스크 영역의 중심 좌표 계산
                contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    # 가장 큰 컨투어를 기준으로 중심점 계산
                    cnt = max(contours, key=cv2.contourArea)
                    M = cv2.moments(cnt)
                    if M["m00"] != 0:
                        cX = int(M["m10"] / M["m00"])
                        cY = int(M["m01"] / M["m00"])
                        # 평균 색상 정보를 텍스트로 표시
                        color_text = f"color : {show_color}"
                        cv2.putText(frame, color_text, (cX - 50, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    # 윤곽선 그리기
                    cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)  # 초록색 선으로 윤곽선 그리기
    # 영상 출력
    cv2.imshow("YOLOv8 Segmentation", frame)
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# 리소스 해제
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 top, 14.2ms
Speed: 1.1ms preprocess, 14.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 top, 13.2ms
Speed: 1.2ms preprocess, 13.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 top, 13.4ms
Speed: 1.7ms preprocess, 13.4ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 top, 13.3ms
Speed: 1.1ms preprocess, 13.3ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 top, 13.3ms
Speed: 1.2ms preprocess, 13.3ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 top, 13.2ms
Speed: 1.0ms preprocess, 13.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 top, 12.2ms
Speed: 2.6ms preprocess, 12.2ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 top, 12.1ms
Speed: 1.9ms preprocess, 12.1ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 to